In [17]:
from Stitchr import stitchrfunctions as fxn
from Stitchr import stitchr as st
import pandas as pd
import wandb
import pandas as pd
from dotenv import load_dotenv
import os
import subprocess
import numpy as np

This is the official documentatin if one would like to use Stitchr in an individual pipeline: 
[Stitchr Doc](https://github.com/JamieHeather/stitchr/blob/main/docs/importing.rst)

In [18]:
chain = "TRB"
species = "HUMAN"

tcr_dat, functionality, partial = fxn.get_imgt_data(chain, st.gene_types, species)
codons = fxn.get_optimal_codons('', species)

V_segment = "TRBV9*01"
J_segment = "TRBJ2-2*01"
CDR3_sequence_beta = "CASSENTANTGELFF"

tcr_bits = {"v": V_segment, "j": J_segment, "cdr3": CDR3_sequence_beta,
            "l": "TRBV7-3*01", "c": "TRBC1*01",
            "skip_c_checks": False, "species": species, "seamless": False,
            "5_prime_seq": "", "3_prime_seq": "", "name": "TCR"}

stitched = st.stitch(tcr_bits, tcr_dat, functionality, partial, codons, 3, "")
stitched

(['TCR',
  'TRBV9*01',
  'TRBJ2-2*01',
  'TRBC1*01',
  'CASSENTANTGELFF',
  'TRBV7-3*01(L)'],
 'ATGGGCACCAGGCTCCTCTGCTGGGCAGCCCTGTGCCTCCTGGGGGCAGATCACACAGATTCTGGAGTCACACAAACCCCAAAGCACCTGATCACAGCAACTGGACAGCGAGTGACGCTGAGATGCTCCCCTAGGTCTGGAGACCTCTCTGTGTACTGGTACCAACAGAGCCTGGACCAGGGCCTCCAGTTCCTCATTCAGTATTATAATGGAGAAGAGAGAGCAAAAGGAAACATTCTTGAACGATTCTCCGCACAACAGTTCCCTGACTTGCACTCTGAACTAAACCTGAGCTCTCTGGAGCTGGGGGACTCAGCTTTGTATTTCTGTGCCAGCAGCGAGAACACCGCCAACACCGGGGAGCTGTTTTTTGGAGAAGGCTCTAGGCTGACCGTACTGGAGGACCTGAACAAGGTGTTCCCACCCGAGGTCGCTGTGTTTGAGCCATCAGAAGCAGAGATCTCCCACACCCAAAAGGCCACACTGGTGTGCCTGGCCACAGGCTTCTTCCCCGACCACGTGGAGCTGAGCTGGTGGGTGAATGGGAAGGAGGTGCACAGTGGGGTCAGCACGGACCCGCAGCCCCTCAAGGAGCAGCCCGCCCTCAATGACTCCAGATACTGCCTGAGCAGCCGCCTGAGGGTCTCGGCCACCTTCTGGCAGAACCCCCGCAACCACTTCCGCTGTCAAGTCCAGTTCTACGGGCTCTCGGAGAATGACGAGTGGACCCAGGATAGGGCCAAACCCGTCACCCAGATCGTCAGCGCCGAGGCCTGGGGTAGAGCAGACTGTGGCTTTACCTCGGTGTCCTACCAGCAAGGGGTCCTGTCTGCCACCATCCTCTATGAGATCCTGCTAGGGAAGGCCACCCTGTATGCTGTGCTGGTCAGCGCCCTTGTGTTGA

However the documentation mentions that for high throughput (as we must use) one should use [*thimble*](https://github.com/JamieHeather/stitchr/blob/main/docs/thimble.rst)

In the follwoing we try to convert a whole subset of the VDJdb using this approach:

In [19]:
load_dotenv()
PROJECT_NAME = os.getenv("MAIN_PROJECT_NAME")
VDJDB = os.getenv("VDJDB_ARTIFACT")
wandb.init(project=PROJECT_NAME, job_type="data-combination")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [20]:
vdjdb_artifact = wandb.use_artifact(f"{VDJDB}:latest")
vdjdb_table = vdjdb_artifact.get(f"{VDJDB}.table.json")
vdjdb_df = pd.DataFrame(data=vdjdb_table.data, columns=vdjdb_table.columns)
vdjdb_df

wandb: Downloading large artifact VDJdb:latest, 79.33MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
1,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2
2,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2
3,3,TRB,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYEPGQVSHYSNQPQHF"", ""cdr3_old"": ""...",2
4,4,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSALASLNEQFF"", ""cdr3_old"": ""CASSAL...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84180,30590,TRA,CMDEGGSNYKLTF,TRAV26-1*01,TRAJ53*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CMDEGGSNYKLTF"", ""cdr3_old"": ""CMDEGGS...",0
84181,30591,TRA,CSLYNNNDMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CSLYNNNDMRF"", ""cdr3_old"": ""CSLYNNNDM...",0
84182,30592,TRA,CALSTDSWGKLQF,TRAV6*01,TRAJ24*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CALSTDSWGKLQF"", ""cdr3_old"": ""CALSTDS...",0
84183,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAPQGATNKLIF"", ""cdr3_old"": ""CAPQGATN...",2


Here we first create only beta-chain,only alpha-chain or only paired-chain datasets and store them in a .tsv!

This first try is by using only paired data! Then we transform the dataset according to the documentation so that it fits the [template for the documentation](https://github.com/JamieHeather/stitchr/tree/main/templates)

In [21]:
only_paired_chains = "../data/VDJdb/onlyPairedStitchr.tsv"

paired_df = pd.read_csv(only_paired_chains, sep="\t")
paired_df

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
1,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2
2,3,TRB,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYEPGQVSHYSNQPQHF"", ""cdr3_old"": ""...",2
3,4,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSALASLNEQFF"", ""cdr3_old"": ""CASSAL...",2
4,5,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEQGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56233,30590,TRA,CMDEGGSNYKLTF,TRAV26-1*01,TRAJ53*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CMDEGGSNYKLTF"", ""cdr3_old"": ""CMDEGGS...",0
56234,30591,TRA,CSLYNNNDMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CSLYNNNDMRF"", ""cdr3_old"": ""CSLYNNNDM...",0
56235,30592,TRA,CALSTDSWGKLQF,TRAV6*01,TRAJ24*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CALSTDSWGKLQF"", ""cdr3_old"": ""CALSTDS...",0
56236,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAPQGATNKLIF"", ""cdr3_old"": ""CAPQGATN...",2


In [22]:
# Splitting the DataFrame into TRB and TRA based on the line transition
line_transition = 28120  # Line where information about beta chains termiante
TRB_df = paired_df.iloc[:line_transition-1].copy()  # TRB chains information
TRA_df = paired_df.iloc[line_transition-1:].copy()  # TRA chains information

# Renaming columns for TRB DataFrame
TRB_df.rename(columns={
    "complex.id": "TCR_name", 
    "V": "TRBV", 
    "J": 'TRBJ', 
    'CDR3': 'TRB_CDR3'
}, inplace=True)

TRA_df.rename(columns={
    'complex.id': 'TCR_name', 
    'V': 'TRAV', 
    'J': 'TRAJ', 
    'CDR3': 'TRA_CDR3'
}, inplace=True)

# Optionally, save these DataFrames to separate files for further use
TRB_df.to_csv('./trb_chains.tsv', sep='\t', index=False)
TRA_df.to_csv('./tra_chains.tsv', sep='\t', index=False)

In [23]:
TRB_df.columns

Index(['TCR_name', 'Gene', 'TRB_CDR3', 'TRBV', 'TRBJ', 'Species', 'MHC A',
       'MHC B', 'MHC class', 'Epitope', 'Epitope gene', 'Epitope species',
       'Reference', 'Method', 'Meta', 'CDR3fix', 'Score'],
      dtype='object')

In [24]:
TRA_df.columns

Index(['TCR_name', 'Gene', 'TRA_CDR3', 'TRAV', 'TRAJ', 'Species', 'MHC A',
       'MHC B', 'MHC class', 'Epitope', 'Epitope gene', 'Epitope species',
       'Reference', 'Method', 'Meta', 'CDR3fix', 'Score'],
      dtype='object')

In [25]:
alpha_df = pd.read_csv('./tra_chains.tsv', sep='\t')
beta_df = pd.read_csv('./trb_chains.tsv', sep='\t')

merged_df_complete = pd.merge(beta_df, alpha_df, on=['TCR_name', "Species", "MHC A", "MHC B", "MHC class", "Epitope", "Epitope gene", "Epitope species"])
merged_df_complete.rename(columns={
    'Gene_x': 'Gene_TRB',
    'Gene_y': 'Gene_TRA',
    'Reference_x': 'Reference_TRB',
    'Method_x': 'Method_TRB',
    'Meta_x': 'Meta_TRB',
    'CDR3fix_x': 'CDR3fix_TRB',
    'Score_x': 'Score_TRB',
    'Reference_y': 'Reference_TRA',
    'Method_y': 'Method_TRA',
    'Meta_y': 'Meta_TRA',
    'CDR3fix_y': 'CDR3fix_TRA',
    'Score_y': 'Score_TRA',
}, inplace=True)
merged_df_complete.to_csv("./mergedAlphaBetaComplete.tsv", sep="\t", index=False)
merged_df_complete.columns

Index(['TCR_name', 'Gene_TRB', 'TRB_CDR3', 'TRBV', 'TRBJ', 'Species', 'MHC A',
       'MHC B', 'MHC class', 'Epitope', 'Epitope gene', 'Epitope species',
       'Reference_TRB', 'Method_TRB', 'Meta_TRB', 'CDR3fix_TRB', 'Score_TRB',
       'Gene_TRA', 'TRA_CDR3', 'TRAV', 'TRAJ', 'Reference_TRA', 'Method_TRA',
       'Meta_TRA', 'CDR3fix_TRA', 'Score_TRA'],
      dtype='object')

In [26]:
merged_df_complete['TRA_leader'] = merged_df_complete['TRAV'] + "(L)"
merged_df_complete['TRB_leader'] = merged_df_complete['TRBV'] + "(L)"

for new_column in ['TRAC', 'TRBC', 'Linker', 'Link_order', 'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq', 'TRB_3_prime_seq']:
    merged_df_complete[new_column] = np.nan

template_columns_order = [
    'TCR_name', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3',
    'TRAC', 'TRBC', 'TRA_leader', 'TRB_leader', 'Linker', 'Link_order',
    'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq', 'TRB_3_prime_seq'
]

df_template = merged_df_complete[template_columns_order]

df_template.to_csv("./TEMPLATE_mergedAlphaBeta.tsv", sep="\t", index=False)

df_template

,TCR_name,TRAV,TRAJ,TRA_CDR3,TRBV,TRBJ,TRB_CDR3,TRAC,TRBC,TRA_leader,TRB_leader,Linker,Link_order,TRA_5_prime_seq,TRA_3_prime_seq,TRB_5_prime_seq,TRB_3_prime_seq
0,1,TRAV26-1*01,TRAJ43*01,CIVRAPGRADMRF,TRBV13*01,TRBJ1-5*01,CASSYLPGQGDHYSNQPQHF,NaN,NaN,TRAV26-1*01(L),TRBV13*01(L),NaN,NaN,NaN,NaN,NaN,NaN
1,2,TRAV20*01,TRAJ28*01,CAVPSGAGSYQLTF,TRBV13*01,TRBJ1-5*01,CASSFEPGQGFYSNQPQHF,NaN,NaN,TRAV20*01(L),TRBV13*01(L),NaN,NaN,NaN,NaN,NaN,NaN
2,3,TRAV2*01,NaN,CAVKASGSRLT,TRBV13*01,TRBJ1-5*01,CASSYEPGQVSHYSNQPQHF,NaN,NaN,TRAV2*01(L),TRBV13*01(L),NaN,NaN,NaN,NaN,NaN,NaN
3,4,TRAV38-2/DV8*01,TRAJ40*01,CAYRPPGTYKYIF,TRBV14*01,TRBJ2-1*01,CASSALASLNEQFF,NaN,NaN,TRAV38-2/DV8*01(L),TRBV14*01(L),NaN,NaN,NaN,NaN,NaN,NaN
4,5,TRAV26-1*01,TRAJ43*01,CIVRAPGRADMRF,TRBV13*01,TRBJ1-5*01,CASSYLPGQGDHYSNQPQHF,NaN,NaN,TRAV26-1*01(L),TRBV13*01(L),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28114,30590,TRAV26-1*01,TRAJ53*01,CMDEGGSNYKLTF,TRBV7-2*01,TRBJ2-3*01,CASSVRSTDTQYF,NaN,NaN,TRAV26-1*01(L),TRBV7-2*01(L),NaN,NaN,NaN,NaN,NaN,NaN
28115,30591,TRAV26-1*01,TRAJ43*01,CSLYNNNDMRF,TRBV7-2*01,TRBJ2-3*01,CASSLRYTDTQYF,NaN,NaN,TRAV26-1*01(L),TRBV7-2*01(L),NaN,NaN,NaN,NaN,NaN,NaN
28116,30592,TRAV6*01,TRAJ24*01,CALSTDSWGKLQF,TRBV7-3*01,TRBJ2-1*01,CASSPGQGGDNEQFF,NaN,NaN,TRAV6*01(L),TRBV7-3*01(L),NaN,NaN,NaN,NaN,NaN,NaN
28117,30593,TRAV12-2*01,TRAJ32*01,CAPQGATNKLIF,TRBV5-1*01,TRBJ2-5*01,CASSLGAGGQETQYF,NaN,NaN,TRAV12-2*01(L),TRBV5-1*01(L),NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_template.columns

Index(['TCR_name', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3',
       'TRAC', 'TRBC', 'TRA_leader', 'TRB_leader', 'Linker', 'Link_order',
       'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq',
       'TRB_3_prime_seq'],
      dtype='object')

Below is the desired schema, according to the documentation! We need to prepare the df to fit into this schema.

In [28]:
template_df = pd.read_csv("./thimble_input_example_TRA-TRB.tsv", sep="\t")
template_df.columns

Index(['TCR_name', 'TRAV', 'TRAJ', 'TRA_CDR3', 'TRBV', 'TRBJ', 'TRB_CDR3',
       'TRAC', 'TRBC', 'TRA_leader', 'TRB_leader', 'Linker', 'Link_order',
       'TRA_5_prime_seq', 'TRA_3_prime_seq', 'TRB_5_prime_seq',
       'TRB_3_prime_seq'],
      dtype='object')

In [29]:
str(template_df.columns) == str(template_df.columns)

True

In [30]:
only_paired_path = "./TEMPLATE_mergedAlphaBeta.tsv"
command = ['thimble', '-in', only_paired_path, '-r', 'AB', "-s", "HUMAN", "-o", "testing"]

In [31]:
result = subprocess.run(command, capture_output=True, text=True)

# Check if the command was executed successfully
if result.returncode == 0:
    print("Command executed successfully.")
    # If you want to print the output
    print("Output:", result.stdout)
else:
    print("Error in command execution.")
    # If you want to print the error
    print("Error:", result.stderr)

Command executed successfully.
Output: Took 26.17 seconds

